## Configurar ambiente de trabalho

### Inicializar o espaço de trabalho

In [1]:
import csv
import os
import sys
from collections.abc import Iterable
from datetime import datetime
from functools import partial
from io import BytesIO, StringIO
from itertools import chain
from pathlib import Path
from urllib.request import urlopen
from zipfile import ZipFile

import log
import pandas as pd
from pandas import DataFrame
from pandas.errors import ParserError

REPO_ROOT = Path(os.getcwd()).parent
DATA_DIR = Path(REPO_ROOT, "data")
IN_DATA_DIR = Path(DATA_DIR, "input")
MID_DATA_DIR = Path(DATA_DIR, "middle")
OUT_DATA_DIR = Path(DATA_DIR, "output")

NOW: datetime = datetime.now().astimezone()

for dirpath in [DATA_DIR, IN_DATA_DIR, MID_DATA_DIR, OUT_DATA_DIR]:
    if not dirpath.is_dir():
        dirpath.mkdir()


# set workspace dir
sys._getframe(1).f_globals["__file__"] = str(Path(".").resolve())

# set logging level
log.init(level=log.INFO)

## Fazer download dos dados históricos do Fala.BR

In [3]:
# Reader functions for FalaBR FOIA historical data
foia_readers_params = dict(
    sep=";",
    header=None,
    cache_dates=True,
    date_parser=partial(
        pd.to_datetime, format="%d/%m/%Y", errors="coerce"
    ),
    na_values=["", 0],
)

In [7]:
requests = pd.concat([
    pd.read_csv(
        path,
        encoding="UTF-16",
        index_col="IdPedido",
        names=[
            "IdPedido",
            "ProtocoloPedido",
            "Esfera",
            "OrgaoDestinatario",
            "Situacao",
            "DataRegistro",
            "ResumoSolicitacao",
            "DetalhamentoSolicitacao",
            "PrazoAtendimento",
            "FoiProrrogado",
            "FoiReencaminhado",
            "FormaResposta",
            "OrigemSolicitacao",
            "IdSolicitante",
            "AssuntoPedido",
            "SubAssuntoPedido",
            "DataResposta",
            "Resposta",
            "Decisao",
            "EspecificacaoDecisao",
        ],
        parse_dates=[
            "DataRegistro",
        ],
        usecols=[
            "IdPedido",
            "OrgaoDestinatario",
            "Situacao",
            "DataRegistro",
            "AssuntoPedido",
            "DetalhamentoSolicitacao",
            "Decisao",
            "EspecificacaoDecisao",
        ],
        lineterminator="\r",
        **foia_readers_params,
    ) for path in IN_DATA_DIR.glob(f"????????_Pedidos_csv_????.csv")
])

requests.index = requests.index.to_series().apply(lambda _id: _id.lstrip("\n") if isinstance(_id, str) else _id)

requests

,OrgaoDestinatario,Situacao,DataRegistro,DetalhamentoSolicitacao,AssuntoPedido,Decisao,EspecificacaoDecisao
IdPedido,,,,,,,
1887837,UFPel – Fundação Universidade Federal de Pelotas,Concluída,2015-07-01,"Prezados, Gostaria de solicitar informações s...",Conduta Docente,Acesso Concedido,Resposta solicitada inserida no Fala.Br
1887842,UFRGS – Universidade Federal do Rio Grande do Sul,Concluída,2015-07-01,"Prezados, Gostaria de solicitar informações s...",Outros em Trabalho,Acesso Concedido,Resposta solicitada inserida no Fala.Br
1887846,"IFPI – Instituto Federal de Educação, Ciência ...",Concluída,2015-07-01,"Prezados, Gostaria de solicitar informações s...",Conduta Docente,Acesso Concedido,Resposta solicitada inserida no Fala.Br
1887851,UFCG – Universidade Federal de Campina Grande,Concluída,2015-07-01,"Prezados, Gostaria de solicitar informações s...",Educação Superior,Acesso Concedido,Informações enviadas por e-mail
1887855,UFF – Universidade Federal Fluminense,Concluída,2015-07-01,"Prezados, Gostaria de solicitar informações s...",Educação Superior,Acesso Concedido,Resposta solicitada inserida no Fala.Br
...,...,...,...,...,...,...,...
3447379,MCIDADANIA - Ministério da Cidadania (Desenvol...,Concluída,2021-07-30,Olá boa tarde gostaria de saber dessa informaç...,Cadastro,Não se trata de solicitação de informação,NaN
3447672,MCIDADANIA - Ministério da Cidadania (Desenvol...,Concluída,2021-07-30,Queria informações sobre o motivo da quarta pa...,Auxílio,Não se trata de solicitação de informação,NaN
3447736,MCIDADANIA - Ministério da Cidadania (Desenvol...,Concluída,2021-07-30,Gostaria de saber porque os desempregados dura...,Auxílio,Não se trata de solicitação de informação,NaN


## Pré-processar textos

In [8]:
import re


def preprocess(text: str) -> str:
    return re.sub(r" {2,}", " ", re.sub(r"\W", " ", text.lower()))


requests.dropna(subset=["DetalhamentoSolicitacao"], inplace=True)
requests["DetalhamentoSolicitacao"] = requests["DetalhamentoSolicitacao"].apply(
    preprocess
)

requests

,OrgaoDestinatario,Situacao,DataRegistro,DetalhamentoSolicitacao,AssuntoPedido,Decisao,EspecificacaoDecisao
IdPedido,,,,,,,
1887837,UFPel – Fundação Universidade Federal de Pelotas,Concluída,2015-07-01,prezados gostaria de solicitar informações sob...,Conduta Docente,Acesso Concedido,Resposta solicitada inserida no Fala.Br
1887842,UFRGS – Universidade Federal do Rio Grande do Sul,Concluída,2015-07-01,prezados gostaria de solicitar informações sob...,Outros em Trabalho,Acesso Concedido,Resposta solicitada inserida no Fala.Br
1887846,"IFPI – Instituto Federal de Educação, Ciência ...",Concluída,2015-07-01,prezados gostaria de solicitar informações sob...,Conduta Docente,Acesso Concedido,Resposta solicitada inserida no Fala.Br
1887851,UFCG – Universidade Federal de Campina Grande,Concluída,2015-07-01,prezados gostaria de solicitar informações sob...,Educação Superior,Acesso Concedido,Informações enviadas por e-mail
1887855,UFF – Universidade Federal Fluminense,Concluída,2015-07-01,prezados gostaria de solicitar informações sob...,Educação Superior,Acesso Concedido,Resposta solicitada inserida no Fala.Br
...,...,...,...,...,...,...,...
3447353,MCIDADANIA - Ministério da Cidadania (Desenvol...,Concluída,2021-07-30,gostaria de saber pq meu auxílio do ano de 202...,Auxílio,Não se trata de solicitação de informação,NaN
3447379,MCIDADANIA - Ministério da Cidadania (Desenvol...,Concluída,2021-07-30,olá boa tarde gostaria de saber dessa informaç...,Cadastro,Não se trata de solicitação de informação,NaN
3447672,MCIDADANIA - Ministério da Cidadania (Desenvol...,Concluída,2021-07-30,queria informações sobre o motivo da quarta pa...,Auxílio,Não se trata de solicitação de informação,NaN


## Atribuir rótulos

In [36]:
requests.reset_index().groupby(["Decisao"])["IdPedido"].count().sort_values(ascending=False).head(15)

Decisao
Acesso Concedido                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [37]:
requests.reset_index().groupby("EspecificacaoDecisao")["IdPedido"].count().sort_values(ascending=False)

EspecificacaoDecisao
Resposta solicitada inserida no Fala.Br                                                             297216
Orientação sobre como encontrar a informação solicitada na Internet ou em publicações existentes     50091
Informações enviadas por e-mail                                                                      26640
Pedido genérico                                                                                       7905
Dados pessoais                                                                                        5804
Parte da informação inexistente                                                                       5259
Informação sigilosa de acordo com legislação específica                                               5168
Parte da informação é de competência de outro órgão/entidade                                          5122
Concedido acesso a sistema corporativo para consulta da informação                                    4568
Processo decisór

In [9]:
label_mapping ={
    "Resposta solicitada inserida no Fala.Br": "acesso_concedido",
    "Informações enviadas por e-mail": "acesso_concedido",
    "Concedido acesso a sistema corporativo para consulta da informação": "acesso_concedido",
    "Parte da informação demandará mais tempo para produção": "acesso_concedido",
    "Informações enviadas pelo correio": "acesso_concedido",
    "Comunicada necessidade de pagamento de custos de postagem e/ou reprodução": "acesso_concedido",
    "Orientação sobre como encontrar a informação solicitada na Internet ou em publicações existentes": 
        # "referencia_transparencia_ativa",
        "acesso_concedido",
    "Pedido genérico": "pedido_generico",
    "Parte do pedido é genérico": "pedido_generico",
    "Dados pessoais": "dados_pessoais",
    "Parte da informação contém dados pessoais": "dados_pessoais",
    # "Parte da informação inexistente": "informacao_inexistente",
    # "Processo decisório em curso": "procedimento_em_curso",
    "Pedido desproporcional ou desarrazoado": "desproporcional_ou_trabalho_adicional",
    "Parte do pedido é desproporcional ou desarrazoado": "desproporcional_ou_trabalho_adicional",
    "Pedido exige tratamento adicional de dados": "desproporcional_ou_trabalho_adicional",
    "Pedido incompreensível": "pedido_incompreensivel",
    "Parte do pedido é incompreensível": "pedido_incompreensivel",
    # "Informação sigilosa classificada conforme a Lei 12.527/2011": "informacao_classificada",
    # "Parte da informação é sigilosa e classificada conforme a Lei 12.527/2011": "informacao_classificada",
    "Parte da informação é sigilosa de acordo com legislação específica": "sigilo_especifico",
}

requests["label"] = requests["EspecificacaoDecisao"].map(label_mapping)

requests.dropna(subset=["label"])

,OrgaoDestinatario,Situacao,DataRegistro,DetalhamentoSolicitacao,AssuntoPedido,Decisao,EspecificacaoDecisao,label
IdPedido,,,,,,,,
1887837,UFPel – Fundação Universidade Federal de Pelotas,Concluída,2015-07-01,prezados gostaria de solicitar informações sob...,Conduta Docente,Acesso Concedido,Resposta solicitada inserida no Fala.Br,acesso_concedido
1887842,UFRGS – Universidade Federal do Rio Grande do Sul,Concluída,2015-07-01,prezados gostaria de solicitar informações sob...,Outros em Trabalho,Acesso Concedido,Resposta solicitada inserida no Fala.Br,acesso_concedido
1887846,"IFPI – Instituto Federal de Educação, Ciência ...",Concluída,2015-07-01,prezados gostaria de solicitar informações sob...,Conduta Docente,Acesso Concedido,Resposta solicitada inserida no Fala.Br,acesso_concedido
1887851,UFCG – Universidade Federal de Campina Grande,Concluída,2015-07-01,prezados gostaria de solicitar informações sob...,Educação Superior,Acesso Concedido,Informações enviadas por e-mail,acesso_concedido
1887855,UFF – Universidade Federal Fluminense,Concluída,2015-07-01,prezados gostaria de solicitar informações sob...,Educação Superior,Acesso Concedido,Resposta solicitada inserida no Fala.Br,acesso_concedido
...,...,...,...,...,...,...,...,...
3445924,"IFSP – Instituto Federal de Educação, Ciência ...",Concluída,2021-07-30,prezado a existe normativa regulamentando a lg...,Dados Pessoais - LGPD,Acesso Concedido,Resposta solicitada inserida no Fala.Br,acesso_concedido
3446156,DPRF – Departamento de Polícia Rodoviária Federal,Concluída,2021-07-30,bom dia venho por meio desta procurar me infor...,Multa,Acesso Concedido,Resposta solicitada inserida no Fala.Br,acesso_concedido
3447132,"INMETRO – Instituto Nacional de Metrologia, Qu...",Concluída,2021-07-30,busco orientação para projeto de transformação...,Acreditação de Organismos e Laboratórios,Acesso Concedido,Informações enviadas por e-mail,acesso_concedido


In [ ]:
requests["DetalhamentoSolicitacao"] = requests.apply(
    lambda row: "__label__{} {}".format(row['label'], row['DetalhamentoSolicitacao']),
    axis=1,
)
requests

## Separar conjuntos de treinamento e de teste

Separar um trimestre aleatório por ano para compor o conjunto de testes, de modo a comparar a performance dos modelos. Utilizar trechos contínuos da série temporal é uma estratégia para tentar garantir que os modelos sejam robustos em relação também a assuntos passageiros, que possam aparecer durante um período curto de tempo sem terem aparecido de forma consistente no conjunto de treinamento.

In [10]:
from random import randint, seed

from imblearn.under_sampling import RandomUnderSampler
# from sklearn.model_selection import train_test_split


seed(0)

labels = set(requests["label"].dropna().unique())
labels.remove("acesso_concedido")


for label in labels:
    
    full_data = requests.query("label == 'acesso_concedido' | label == @label")

    test_data = pd.DataFrame()
    for dt in pd.date_range(min(full_data["DataRegistro"]), max(full_data["DataRegistro"]), freq="YS"):
        test_dt_start = dt + pd.Timedelta(days=randint(0, 365-90))
        test_dt_end = test_dt_start + pd.Timedelta(days=90)
        test_data = pd.concat([test_data, full_data.query("@test_dt_start <= DataRegistro < @test_dt_end")])

    train_data = full_data.loc[~full_data.index.isin(test_data.index)]
    under_sampler = RandomUnderSampler(random_state=0)
    train_data_resampled, _ = under_sampler.fit_resample(train_data.drop(columns=["label"]), train_data["label"])
    train_data_resampled

    # stratify_cols = ["OrgaoDestinatario", "AssuntoPedido"]
    # test_size = 0.2

    # positive_cases = requests.query("label == 'acesso_concedido'")
    # train_positive, test_positive = train_test_split(
    #     positive_cases,
    #     test_size=test_size,
    #     stratify=positive_cases[stratify_cols],
    #     random_state=0,
    # )
    
    # negative_cases = requests.query("label == '{}'".format(label))
    # train_negative, test_negative = train_test_split(
    #     positive_cases,
    #     test_size=test_size,
    #     stratify=negative_cases[stratify_cols],
    #     random_state=0,
    # )

    # train = train_negative
    # test = pd.concat([test_negative, test_positive])

    # negative_strata = (
    #     negative_cases
    #     .reset_index()
    #     .groupby(stratify_cols)
    #     ["IdPedido"]
    #     .count()
    #     .rename("NumPedidos")
    #     .reset_index()
    # )

    # for stratum in negative_strata.itertuples():
    #     conditions = ["{} == {}".format(col, stratum[col]) for col in stratify_cols]
    #     condition_expr = " & ".join(conditions)
    #     train = pd.concat([
    #         train,
    #         train_positive.query(condition_expr).sample(stratum["NumPedidos"], random_state=0)
    #     ])

    with open(Path(MID_DATA_DIR, "{}.train.txt".format(label)), "w") as train_file:
        train_file.write("\n".join(train_data_resampled["DetalhamentoSolicitacao"]))

    with open(Path(MID_DATA_DIR, "{}.test.txt".format(label)), "w") as test_file:
        test_file.write("\n".join(test_data["DetalhamentoSolicitacao"]))

## Executar treinamento

In [38]:
import fasttext

model_generic_request = fasttext.train_supervised(
    input=str(Path(MID_DATA_DIR, "pedido_generico.train.txt")),
    autotuneValidationFile=str(Path(MID_DATA_DIR, "pedido_generico.test.txt")),
    autotuneDuration=1800,
)

display(
    model_generic_request.test(str(Path(MID_DATA_DIR, "pedido_generico.test.txt")), k=1)
)

Progress: 100.0% Trials:   64 Best score:  0.935730 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  36388
Number of labels: 2
Progress: 100.0% words/sec/thread:  148075 lr:  0.000000 avg.loss:  0.610074 ETA:   0h 0m 0s


(97961, 0.6013821827053624, 0.6013821827053624)

In [39]:
import fasttext

model_personal_data = fasttext.train_supervised(
    input=str(Path(MID_DATA_DIR, "dados_pessoais.train.txt")),
    autotuneValidationFile=str(Path(MID_DATA_DIR, "dados_pessoais.test.txt")),
    autotuneDuration=1800,
)

display(
    model_personal_data.test(str(Path(MID_DATA_DIR, "dados_pessoais.test.txt")), k=1)
)

Progress: 100.0% Trials:   23 Best score:  0.868786 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  33935
Number of labels: 2
Progress: 100.0% words/sec/thread:   29966 lr:  0.000000 avg.loss:  0.401843 ETA:   0h 0m 0s


(97977, 0.8017289772089368, 0.8017289772089368)

In [40]:
import fasttext

model_additional_labor = fasttext.train_supervised(
    input=str(Path(MID_DATA_DIR, "desproporcional_ou_trabalho_adicional.train.txt")),
    autotuneValidationFile=str(Path(MID_DATA_DIR, "desproporcional_ou_trabalho_adicional.test.txt")),
    autotuneDuration=1800,
)

display(
    model_additional_labor.test(str(Path(MID_DATA_DIR, "desproporcional_ou_trabalho_adicional.test.txt")), k=1)
)

Progress: 100.0% Trials:   11 Best score:  0.841173 ETA:   0h 0m 0s
Training again with best arguments
Read 1M words
Number of words:  41442
Number of labels: 2
Progress: 100.0% words/sec/thread:  813879 lr:  0.000000 avg.loss:  0.648182 ETA:   0h 0m 0s


(97836, 0.7257246821211006, 0.7257246821211006)

In [41]:
import fasttext

model_not_understandable = fasttext.train_supervised(
    input=str(Path(MID_DATA_DIR, "pedido_incompreensivel.train.txt")),
    autotuneValidationFile=str(Path(MID_DATA_DIR, "pedido_incompreensivel.test.txt")),
    autotuneDuration=1800,
)

display(
    model_not_understandable.test(str(Path(MID_DATA_DIR, "pedido_incompreensivel.test.txt")), k=1)
)

Progress: 100.0% Trials:  118 Best score:  0.971516 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  23694
Number of labels: 2
Progress: 100.0% words/sec/thread: 1516727 lr:  0.000000 avg.loss:  0.459001 ETA:   0h 0m 0s


(96685, 0.7119098102084087, 0.7119098102084087)

In [52]:
import fasttext

model_specific_classification = fasttext.train_supervised(
    input=str(Path(MID_DATA_DIR, "sigilo_especifico.train.txt")),
    # autotuneValidationFile=str(Path(MID_DATA_DIR, "sigilo_especifico.test.txt")),
    # autotuneDuration=1800,
)

display(
    model_specific_classification.test(str(Path(MID_DATA_DIR, "sigilo_especifico.test.txt")), k=1)
)

Read 0M words
Number of words:  21147
Number of labels: 2
Progress: 100.0% words/sec/thread: 1133277 lr:  0.000000 avg.loss:  0.589484 ETA:   0h 0m 0s


(96613, 0.6420771531781437, 0.6420771531781437)

In [43]:
model_generic_request.save_model(str(Path(OUT_DATA_DIR, "pedido_generico.bin")))
model_personal_data.save_model(str(Path(OUT_DATA_DIR, "dados_pessoais.bin")))
model_additional_labor.save_model(str(Path(OUT_DATA_DIR, "desproporcional_ou_trabalho_adicional.bin")))
model_not_understandable.save_model(str(Path(OUT_DATA_DIR, "pedido_incompreensivel.bin")))
model_specific_classification.save_model(str(Path(OUT_DATA_DIR, "sigilo_especifico.bin")))

### Validate results

In [44]:
list(IN_DATA_DIR.glob(f"????????_Pedidos_csv_2021.csv"))

[]

In [49]:
url = (
    "https://dadosabertos-download.cgu.gov.br/FalaBR"
    "/Arquivos_FalaBR_Filtrado/Arquivos_csv_2021.zip"
)
http_response = urlopen(url)
zipfile = ZipFile(BytesIO(http_response.read()))
zipfile.extractall(path=IN_DATA_DIR)
requests_2021_path = Path(
    IN_DATA_DIR,
    "{:%Y%m%d}_Pedidos_csv_2021.csv".format(NOW),
)

requests_2021 = pd.read_csv(
    requests_2021_path,
    encoding="UTF-16",
    index_col="IdPedido",
    names=[
        "IdPedido",
        "ProtocoloPedido",
        "Esfera",
        "OrgaoDestinatario",
        "Situacao",
        "DataRegistro",
        "ResumoSolicitacao",
        "DetalhamentoSolicitacao",
        "PrazoAtendimento",
        "FoiProrrogado",
        "FoiReencaminhado",
        "FormaResposta",
        "OrigemSolicitacao",
        "IdSolicitante",
        "AssuntoPedido",
        "SubAssuntoPedido",
        "DataResposta",
        "Resposta",
        "Decisao",
        "EspecificacaoDecisao",
    ],
    parse_dates=[
        "DataRegistro",
    ],
    usecols=[
        "IdPedido",
        "OrgaoDestinatario",
        "Situacao",
        "DataRegistro",
        "AssuntoPedido",
        "DetalhamentoSolicitacao",
        "Decisao",
        "EspecificacaoDecisao",
    ],
    lineterminator="\r",
    **foia_readers_params,
)

requests_2021.index = requests_2021.index.to_series().apply(lambda _id: _id.lstrip("\n") if isinstance(_id, str) else _id)


requests_2021.dropna(subset=["DetalhamentoSolicitacao"], inplace=True)
requests_2021["DetalhamentoSolicitacao"] = requests_2021["DetalhamentoSolicitacao"].apply(
    preprocess
)

requests_2021["label"] = requests_2021["EspecificacaoDecisao"].map(label_mapping)
requests_2021.dropna(subset=["label"])

requests_2021

,OrgaoDestinatario,Situacao,DataRegistro,DetalhamentoSolicitacao,AssuntoPedido,Decisao,EspecificacaoDecisao,label
IdPedido,,,,,,,,
2793736,EBSERH - HUAB-UFRN - Hospital Universitário An...,Concluída,2021-01-01,solicito esclarecimentos do processo de seleçã...,Outros em Administração,Acesso Concedido,Resposta solicitada inserida no Fala.Br,acesso_concedido
2793760,SUSEP – Superintendência de Seguros Privados,Concluída,2021-01-01,por favor referente processo susep 15414633999...,Outros em Previdência,Acesso Concedido,Resposta solicitada inserida no Fala.Br,acesso_concedido
2793766,UFMG – Universidade Federal de Minas Gerais,Concluída,2021-01-01,bom dia tenho interesse de saber se a universi...,Concurso,Acesso Concedido,Resposta solicitada inserida no Fala.Br,acesso_concedido
2793772,"ANP – Agência Nacional do Petróleo, Gás Natura...",Concluída,2021-01-01,solicito da anp os relatórios de fiscalização ...,Acesso à informação,Acesso Negado,Informação sigilosa classificada conforme a Le...,NaN
2793784,MJSP – Ministério da Justiça e Segurança Pública,Concluída,2021-01-01,excelentíssimos meus cumprimentos venho por me...,Acesso à informação,Acesso Concedido,Resposta solicitada inserida no Fala.Br,acesso_concedido
...,...,...,...,...,...,...,...,...
3447353,MCIDADANIA - Ministério da Cidadania (Desenvol...,Concluída,2021-07-30,gostaria de saber pq meu auxílio do ano de 202...,Auxílio,Não se trata de solicitação de informação,NaN,NaN
3447379,MCIDADANIA - Ministério da Cidadania (Desenvol...,Concluída,2021-07-30,olá boa tarde gostaria de saber dessa informaç...,Cadastro,Não se trata de solicitação de informação,NaN,NaN
3447672,MCIDADANIA - Ministério da Cidadania (Desenvol...,Concluída,2021-07-30,queria informações sobre o motivo da quarta pa...,Auxílio,Não se trata de solicitação de informação,NaN,NaN


In [56]:
models = {
    "pedido_generico": model_generic_request,
    "dados_pessoais": model_personal_data,
    "desproporcional_ou_trabalho_adicional": model_additional_labor,
    "pedido_incompreensivel": model_not_understandable,
    "sigilo_especifico": model_specific_classification,
}

for label in labels:
    requests_2021[label] = requests_2021["DetalhamentoSolicitacao"].apply(
        lambda text: models[label].predict(text)[0][0].lstrip("__label__")
    )

In [67]:
for label in labels:
    requests_2021.loc[requests_2021["label"]==label, "acerto"] = requests_2021.apply(
        lambda row: row[label] == row["label"],
        axis=1,
    )

requests_2021.loc[requests_2021["label"]=="acesso_concedido", "acerto"] = (
    requests_2021.loc[requests_2021["label"]=="acesso_concedido"].apply(
        lambda row: all(row[label].endswith("cesso_concedido") for label in labels),
        axis=1,
    )
)

In [68]:
requests_2021.groupby("label").agg({"acerto": lambda group: sum(group)/len(group)})

,acerto
label,
acesso_concedido,0.233635
dados_pessoais,0.639194
desproporcional_ou_trabalho_adicional,0.663317
pedido_generico,0.790072
pedido_incompreensivel,0.734177
sigilo_especifico,0.677083


## Com Spacy + auto-sklearn

In [13]:
import spacy

nlp = spacy.load("pt_core_news_lg")

In [36]:
import dask.dataframe as dd

X = dd.from_pandas(
    requests["DetalhamentoSolicitacao"], chunksize=1024, sort=False
)
X

Dask Series Structure:
npartitions=579
    object
       ...
     ...  
       ...
       ...
Name: DetalhamentoSolicitacao, dtype: object
Dask Name: from_pandas, 579 tasks

In [45]:
embeddings = X.apply(lambda doc: pd.Series(nlp(doc).vector))
embeddings = embeddings.compute()
embeddings

In [56]:
embeddings.to_hdf(MID_DATA_DIR / "features.hd5", key="X")
requests["label"].to_hdf(MID_DATA_DIR / "labels.hd5", key="y")

INFO: numexpr.utils: NumExpr defaulting to 8 threads.


In [58]:
from sklearn.model_selection import train_test_split

X = pd.read_hdf(MID_DATA_DIR / "features.hd5", key="X")
y = pd.read_hdf(MID_DATA_DIR / "labels.hd5", key="y")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1
)

In [60]:
X_train.to_hdf(MID_DATA_DIR / "features.train.hd5", key="X_train")
X_test.to_hdf(MID_DATA_DIR / "features.test.hd5", key="X_test")
y_train.to_hdf(MID_DATA_DIR / "labels.train.hd5", key="y_train")
y_test.to_hdf(MID_DATA_DIR / "labels.test.hd5", key="y_test")

In [2]:
X_train = pd.read_hdf(MID_DATA_DIR / "features.train.hd5", key="X_train")
X_test = pd.read_hdf(MID_DATA_DIR / "features.test.hd5", key="X_test")
y_train = pd.read_hdf(MID_DATA_DIR / "labels.train.hd5", key="y_train")
y_test = pd.read_hdf(MID_DATA_DIR / "labels.test.hd5", key="y_test")

In [5]:
X_train.shape

(363766, 300)

In [6]:
from autosklearn.experimental.askl2 import AutoSklearnClassifier

automl = AutoSklearnClassifier(
    # time_left_for_this_task=3600,
    # per_run_time_limit=300,
    n_jobs=4,
    tmp_folder=str(MID_DATA_DIR / "model_training_tmp5"),
    memory_limit=5120,
)
automl.fit(X_train, y_train)

terminate called after throwing an instance of 'std::bad_alloc'
  what():  std::bad_alloc
[WARNING] [2021-08-10 06:21:09,558:Client-autosklearn.util.backend] Directory /home/bcbernardo/Projetos/pessoais/analaizer/data/middle/model_training_tmp5/.auto-sklearn/ensembles does not exist
[WARNING] [2021-08-10 06:21:09,559:Client-AutoML(1):80f80b27-f974-11eb-9508-24ee9a6548e3] No valid ensemble was created. Please check the logfile for errors. Default to the best individual estimator:[(1, 5, 0.0)]


AutoSklearnClassifier(memory_limit=5120, n_jobs=4, per_run_time_limit=1440,
                      tmp_folder='/home/bcbernardo/Projetos/pessoais/analaizer/data/middle/model_training_tmp5')

In [9]:
import pickle

with open(OUT_DATA_DIR / "model.pickle", "wb") as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)

In [14]:
# from sklearn.metrics import (
#     accuracy_score,
#     f1_score,
#     roc_auc_score,
#     zero_one_loss,
# )

# y_test = y_test.dropna()
# X_test = X_test.loc[y_test.index]

# y_hat = automl.predict(X_test)
print("F1 score", f1_score(y_test, y_hat, average="weighted"))
# print("ROC AUC", roc_auc_score(y_test, y_hat))
print("Zero one loss", zero_one_loss(y_test, y_hat))

F1 score 0.8886802354694061
Zero one loss 0.07863521111233607


In [10]:
print(automl.leaderboard())

          rank  ensemble_weight type      cost      duration
model_id                                                    
5            1              1.0  mlp  0.080455  30032.183494


In [11]:
print(automl.show_models())

[(1.000000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'classifier:__choice__': 'mlp', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'minority_coalescer', 'data_preprocessing:numerical_transformer:imputation:strategy': 'most_frequent', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'quantile_transformer', 'feature_preprocessor:__choice__': 'feature_agglomeration', 'classifier:mlp:activation': 'tanh', 'classifier:mlp:alpha': 0.00012344934870642978, 'classifier:mlp:batch_size': 'auto', 'classifier:mlp:beta_1': 0.9, 'classifier:mlp:beta_2': 0.999, 'classifier:mlp:early_stopping': 'train', 'classifier:mlp:epsilon': 1e-08, 'classifier:mlp:hidden_layer_depth': 1, 'classifier:mlp:learning_rate_init': 0.0010302058554176633, 'classifier:mlp:n_iter_no_change': 32, 'classifier:mlp:num_nodes_per_layer': 32, 'classifier:mlp:shuffle': 'T

In [30]:
comparison = pd.concat([y_test.rename("verificado").reset_index(), pd.Series(y_hat, name="predito")], axis=1)
comparison

,IdPedido,verificado,predito
0,2148392,pedido_generico,acesso_concedido
1,2980435,acesso_concedido,acesso_concedido
2,1536823,acesso_concedido,acesso_concedido
3,2026836,acesso_concedido,acesso_concedido
4,1502214,dados_pessoais,acesso_concedido
...,...,...,...
90705,1869528,acesso_concedido,acesso_concedido
90706,1505034,acesso_concedido,acesso_concedido
90707,2372655,acesso_concedido,acesso_concedido
90708,2019335,acesso_concedido,acesso_concedido


In [35]:
comparison.query("verificado=='pedido_generico'").apply(lambda row: row["verificado"]==row["predito"], axis=1).agg((sum, "count"))

sum        27
count    2009
dtype: int64

In [36]:
comparison.query("verificado=='dados_pessoais'").apply(lambda row: row["verificado"]==row["predito"], axis=1).agg((sum, "count"))

sum       158
count    1759
dtype: int64

In [37]:
comparison.query("verificado=='pedido_incompreensivel'").apply(lambda row: row["verificado"]==row["predito"], axis=1).agg((sum, "count"))

sum       10
count    868
dtype: int64

In [38]:
comparison.query("verificado=='desproporcional_ou_trabalho_adicional'").apply(lambda row: row["verificado"]==row["predito"], axis=1).agg((sum, "count"))

sum        77
count    1961
dtype: int64

In [39]:
comparison.query("verificado=='sigilo_especifico'").apply(lambda row: row["verificado"]==row["predito"], axis=1).agg((sum, "count"))

sum       13
count    600
dtype: int64

In [40]:
comparison.query("verificado=='acesso_concedido'").apply(lambda row: row["verificado"]==row["predito"], axis=1).agg((sum, "count"))

sum      83292
count    83513
dtype: int64

In [41]:
comparison.query("verificado!='acesso_concedido'").apply(lambda row: row["verificado"]==row["predito"], axis=1).agg((sum, "count"))

sum       285
count    7197
dtype: int64